### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import locale as locale
import numpy as np

In [2]:
locale.setlocale(locale.LC_ALL, '')
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data = purchase_data.set_index('Purchase ID')
#test if the data is good
print(len(purchase_data.dropna( )) == len(purchase_data))
purchase_data['Gender'].value_counts( )

True


Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64

In [3]:
locale.localeconv( )

{'int_curr_symbol': 'USD',
 'currency_symbol': '$',
 'mon_decimal_point': '.',
 'mon_thousands_sep': ',',
 'mon_grouping': [3, 0],
 'positive_sign': '',
 'negative_sign': '-',
 'int_frac_digits': 2,
 'frac_digits': 2,
 'p_cs_precedes': 1,
 'p_sep_by_space': 0,
 'n_cs_precedes': 1,
 'n_sep_by_space': 0,
 'p_sign_posn': 3,
 'n_sign_posn': 0,
 'decimal_point': '.',
 'thousands_sep': ',',
 'grouping': [3, 0]}

In [4]:

m = round(purchase_data['Price'].mean( ))
print(locale.currency(m))

$3.00


In [5]:
purchase_data.dtypes


SN            object
Age            int64
Gender        object
Item ID        int64
Item Name     object
Price        float64
dtype: object

## Player Count

* Display the total number of players


In [6]:
tp = len(purchase_data['SN'].unique( ))
pd.DataFrame({'Total Players': [ tp ] })


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [7]:

n_items = len( purchase_data['Item Name'].unique( ) )
n_purchases =  len(purchase_data)
n_revenue = purchase_data['Price'].sum( )
avg_sale = purchase_data['Price'].mean( )

purch_summary_df = pd.DataFrame( {
    'Number of Unique Items':[ n_items ],
    'Average Price':[ locale.currency( avg_sale ) ],
    'Number of Purchases':[ n_purchases ],
    'Total Revenue':[ locale.currency( n_revenue ) ]
})

purch_summary_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
#did anyone change gender?


id_gb = purchase_data.groupby(['SN'])
id_gender_gb = purchase_data.groupby(['SN','Gender'])
if len(id_gender_gb) == len(id_gb):
    print("All cis")
else:
    print("Trans Lives Matter")
    
#or have a birthday?
id_age_gb = purchase_data.groupby(['SN','Age'])
if len(id_age_gb) == len(id_gb):
    print("I'll stop time")
else:
    print("Happy birthday to " + str(len(id_age_gb) - len(id_gb)) + " players!")

All cis
I'll stop time


In [9]:
#since our game is too new for any players to have changed their settings, we can collapse the user data:


In [10]:
total_sales_by_sn = id_gb.sum( )['Price']
total_purchase_by_user = pd.DataFrame(total_sales_by_sn).rename( columns={'Price': 'Total Purchase'})
total_purchase_by_user

,Total Purchase
SN,
Adairialis76,2.28
Adastirin33,4.48
Aeda94,4.91
Aela59,4.32
Aelaria33,1.79
...,...
Yathecal82,6.22
Yathedeu43,6.02
Yoishirrala98,4.58


In [11]:
#total_sales_by_sn[id_gb.count()['Gender'] > 1]
#this does not count the sales by trans users

In [12]:
#total_sales_by_sn[id_gb.count()['Gender'] == 1].sum()

In [13]:
#id_gb.count()['Gender'].value_counts()

In [14]:
#player_gb = purchase_data.groupby(['SN','Gender','Age'])['Gender']
#player_gb.count()

In [15]:
user_data = purchase_data.groupby(['SN']).first( )
user_data = user_data[['Gender','Age']]
user_data.head( )

,Gender,Age
SN,,
Adairialis76,Male,16
Adastirin33,Female,35
Aeda94,Male,17
Aela59,Male,21
Aelaria33,Male,23


In [16]:
gender_gb = purchase_data.groupby(['Gender'])
gender_total_sales = gender_gb['Price'].sum( )
gender_total_sales = pd.DataFrame(gender_total_sales)
gender_total_sales = gender_total_sales.rename( columns={'Price': 'Total Purchase Value'})
gender_total_sales 

,Total Purchase Value
Gender,
Female,361.94
Male,1967.64
Other / Non-Disclosed,50.19


In [17]:
sale_count_by_gender = gender_gb.count( )['SN']
sale_count_by_gender = pd.DataFrame(sale_count_by_gender)
sale_count_by_gender = sale_count_by_gender.rename( columns={'SN':'Purchase Count'})
sale_count_by_gender

,Purchase Count
Gender,
Female,113
Male,652
Other / Non-Disclosed,15


In [18]:
users_by_gender = user_data.groupby('Gender').count( ).rename( columns={'Age': 'Number of Users'})
users_by_gender 

,Number of Users
Gender,
Female,81
Male,484
Other / Non-Disclosed,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [19]:
average_sale_by_gender = gender_total_sales['Total Purchase Value'] / sale_count_by_gender['Purchase Count']
average_sale_by_gender

Gender
Female                   3.203009
Male                     3.017853
Other / Non-Disclosed    3.346000
dtype: float64

In [20]:
gender_summary = sale_count_by_gender
gender_summary['Average Purchase Price'] = average_sale_by_gender.map(locale.currency)

gender_summary = pd.merge(gender_summary,gender_total_sales,on='Gender')

gender_summary['Avg Total Purchase per Person'] = (
    gender_total_sales['Total Purchase Value'] / users_by_gender['Number of Users']
).map(locale.currency)

gender_summary['Total Purchase Value'] = gender_summary['Total Purchase Value'].map(
    lambda c: locale.currency(c, grouping=True)
)

gender_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

